In [89]:
import pandas as pd
import numpy as np
import pyodbc
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import xgboost as xgb
import lightgbm as lgb


In [90]:
server = '192.168.1.212'
database = 'master'
username = 'test'
password = 'tester2024'

mssql_conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};'

In [93]:
msql_query = f"""
        select *
        from ProjectNew..FullCompanyInfo
        """
print(msql_query)


        select *
        from ProjectNew..FullCompanyInfo
        


In [94]:
try:
    mssql_conn = pyodbc.connect(mssql_conn_str)
    print("Kết nối cơ sở dữ liệu thành công")
except pyodbc.Error as e:
    print(f"Lỗi khi kết nối cơ sở dữ liệu: {e}")

data = pd.read_sql_query(msql_query, mssql_conn)

mssql_conn.close()

Kết nối cơ sở dữ liệu thành công


C:\Users\nnson\AppData\Local\Temp\ipykernel_18816\1879818091.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(msql_query, mssql_conn)


In [98]:
def processing_data(df1):
    df = df1.copy()

    # One-hot encoding cho cột CompanyType
    df = pd.get_dummies(df, columns=['CompanyType'], prefix='Type')
        
    if 'CompanyId' in df.columns:
        df.drop(columns=['CompanyId'], inplace=True)

    # for id in range(11, 25):
    #     mean_col_name = f'{id}_mean'
        
    #     # Tìm tất cả các cột tương ứng với id hiện tại
    #     cols_to_avg = [f'{year}_{id}' for year in range(2015, 2022)]
        
    #     # Tính giá trị trung bình và thêm vào cột mới
    #     df[mean_col_name] = df[cols_to_avg].mean(axis=1)

    # # Xóa các cột cũ
    # df.drop(columns=[f'{year}_{id}' for year in range(2015, 2022) for id in range(11, 25)], inplace=True)

    # Normalize các cột còn lại với giá trị từ 0 đến 1
    scaler = MinMaxScaler()
    df[df.columns] = scaler.fit_transform(df[df.columns])
    
    # Thay thế tất cả các giá trị NaN trong df thành 0
    df.fillna(0, inplace=True)
    
    return df

In [99]:
processed_data = processing_data(data)

In [100]:
processed_data

,CompanyAge,FDI,CapitalAmount,NumberOfLabors,Region,FS11,FS12,FS13,FS14,FS15,...,FS20,FS21,FS22,FS23,FS24,Status,Type_LLC1,Type_LLC2,Type_PE,Type_SC
0,0.176471,0.0,0.000292,0.000061,1.0,0.066675,0.000000,0.135581,0.023858,0.061939,...,0.071832,0.077137,0.033333,0.182198,0.217620,0.0,0.0,0.0,0.0,1.0
1,0.382353,0.0,0.000097,0.000076,0.0,0.066634,0.000000,0.135614,0.023858,0.061955,...,0.071798,0.077136,0.033332,0.182198,0.217620,1.0,0.0,0.0,0.0,1.0
2,0.147059,0.0,0.000015,0.000046,1.0,0.066630,0.000000,0.135581,0.023858,0.061939,...,0.071794,0.077136,0.033332,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
3,0.147059,0.0,0.000048,0.000015,1.0,0.066635,0.000000,0.135629,0.023858,0.061945,...,0.071799,0.077144,0.033334,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
4,0.088235,0.0,0.000002,0.000153,1.0,0.066627,0.000000,0.135581,0.023858,0.061939,...,0.071792,0.077136,0.033332,0.182198,0.217620,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235649,0.235294,0.0,0.000049,0.000153,1.0,0.066638,0.000000,0.000000,0.000000,0.000000,...,0.071801,0.000000,0.000000,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0
235650,0.117647,0.0,0.000005,0.000015,1.0,0.066627,0.000000,0.135581,0.023858,0.061939,...,0.071792,0.077136,0.033332,0.182198,0.217620,1.0,0.0,1.0,0.0,0.0
235651,0.205882,0.0,0.000243,0.000153,1.0,0.066642,0.096372,0.135581,0.023917,0.061944,...,0.071841,0.077144,0.033339,0.182202,0.217624,1.0,1.0,0.0,0.0,0.0
235652,0.088235,0.0,0.000024,0.000015,1.0,0.066632,0.096354,0.135581,0.023858,0.061939,...,0.071796,0.077136,0.033332,0.182198,0.217620,1.0,1.0,0.0,0.0,0.0


In [102]:
def train_logistic_regression_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo mô hình Logistic Regression
    model = LogisticRegression()

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Huấn luyện mô hình trên tập train
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred))

In [103]:
train_logistic_regression_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7954256131715183
Training on fold 2...
Accuracy for fold 2: 0.7940252906730034
Training on fold 3...
Accuracy for fold 3: 0.7948739709751337
Training on fold 4...
Accuracy for fold 4: 0.7954680471866248
Training on fold 5...
Accuracy for fold 5: 0.7959685974962869
Training on fold 6...
Accuracy for fold 6: 0.795374496074687
Training on fold 7...
Accuracy for fold 7: 0.7963505198387439
Training on fold 8...
Accuracy for fold 8: 0.7940165499681732
Training on fold 9...
Accuracy for fold 9: 0.7948652662847443
Training on fold 10...
Accuracy for fold 10: 0.7959261616804583

Final Report:
Mean Accuracy: 0.7952294513349374
Standard Deviation of Accuracy: 0.0007473360709186283

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.38      0.00      0.01      4796
         1.0       0.80      1.00      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.59

In [104]:
def train_xgboost_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình XGBoost trên tập train
        model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [105]:
train_xgboost_kfold(processed_data, 10)

Training on fold 1...
Accuracy for fold 1: 0.7997963167274887
Training on fold 2...
Accuracy for fold 2: 0.8000084868030213
Training on fold 3...
Accuracy for fold 3: 0.8011542052108971
Training on fold 4...
Accuracy for fold 4: 0.8025120936943053
Training on fold 5...
Accuracy for fold 5: 0.8032675578187991
Training on fold 6...
Accuracy for fold 6: 0.8029705071079991
Training on fold 7...
Accuracy for fold 7: 0.8013579461065139
Training on fold 8...
Accuracy for fold 8: 0.8004243581582856
Training on fold 9...
Accuracy for fold 9: 0.8014852535539996
Training on fold 10...
Accuracy for fold 10: 0.8015276893698281

Final Report:
Mean Accuracy: 0.8014504414551137
Standard Deviation of Accuracy: 0.001127444727467978

Classification Report for fold 1:
              precision    recall  f1-score   support

         0.0       0.59      0.08      0.15      4796
         1.0       0.81      0.99      0.89     18769

    accuracy                           0.80     23565
   macro avg       0.70

In [106]:
def train_lightgbm_kfold(processed_data, k=5):
    df = processed_data.copy()
    warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
    
    # Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
    X = df.drop('Status', axis=1)
    y = df['Status']

    # Khởi tạo KFold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    accuracies = []
    reports = []

    fold = 1
    for train_index, test_index in kf.split(X):
        print(f"Training on fold {fold}...")
        
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Khởi tạo và huấn luyện mô hình LightGBM trên tập train
        model = lgb.LGBMClassifier()
        model.fit(X_train, y_train)
        
        # Dự đoán trên tập test
        y_pred = model.predict(X_test)
        
        # Đánh giá mô hình
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        reports.append(report)

        print(f"Accuracy for fold {fold}: {accuracy}")
        fold += 1

    # Tính toán và in ra báo cáo cuối cùng
    print("\nFinal Report:")
    print(f"Mean Accuracy: {np.mean(accuracies)}")
    print(f"Standard Deviation of Accuracy: {np.std(accuracies)}")
    
    # In ra classification report cho từng fold
    for i, report in enumerate(reports):
        print(f"\nClassification Report for fold {i+1}:")
        print(classification_report(y.iloc[test_index], y_pred, zero_division=0))


In [107]:
train_lightgbm_kfold(processed_data, 10)

Training on fold 1...
[LightGBM] [Info] Number of positive: 168722, number of negative: 43366
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012842 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4126
[LightGBM] [Info] Number of data points in the train set: 212088, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.795528 -> initscore=1.358577
[LightGBM] [Info] Start training from score 1.358577
Accuracy for fold 1: 0.8003903929389798
Training on fold 2...
[LightGBM] [Info] Number of positive: 168779, number of negative: 43309
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003732 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4125
[LightGBM] [Info] Number of data points in the train set: 212088, number of used features: